In [2]:
import pandas as pd
import os

import re 
import numpy as np
import cv2
from matplotlib.pyplot import imread
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
from zipfile import ZipFile
import shutil
cwd = os.getcwd()
cwd

'/home/tf/HAR-UP/dataset'

In [3]:
import os
import sys
import argparse
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from MODNet.src.models.modnet import MODNet

In [4]:
import cv2
import torch
from PIL import Image, ImageOps
from torchvision import transforms
from matplotlib import pyplot as plt

In [3]:



def load_img(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam , DesiredWidth = 64, DesiredHeight = 64 ):
    IMG = []
    count = 0
    name_img = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)

        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):
                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                for cam_ in range(start_cam , end_cam + 1) :
                    cam = 'Camera'+ str(cam_)

                    with ZipFile( sub + '/' + act  + '/'+ trial +  '/' + sub + act  + trial + cam + '.zip', 'r') as zipObj:
                        zipObj.extractall('CAMERA/' + sub + act  + trial  + cam)

                    for root, dirnames, filenames in os.walk('CAMERA/' + sub + act + trial  +cam ):
                        for filename in filenames:
                            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                                filepath = os.path.join(root, filename)
                                count += 1 
                                if count % 5000 == 0 :
                                    print('{} : {} ' .format(filepath,count))
                                if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png' :
                                    print('----------------------------NO SHAPE---------------------------------')
                                    continue
                                elif len(filepath) > 70 :
                                    print(' {} : Invalid image'.format(filepath))
                                    continue 
                                name_img.append(filepath)
#                                 print('read file')
#                                 img = cv2.imread(filepath, 0) 
                                img = Image.open(filepath)
                                img = np.array(img).astype('uint8')
                                resized = ResizeImage(img, DesiredWidth, DesiredHeight)

                                IMG.append(resized)
                                
                    shutil.rmtree('CAMERA/'+ sub + act + trial + cam)
    return IMG , name_img


In [5]:

def handle_name(path_name) :
    img_name = []
    for path in path_name :
        if len(path) == 68: 
            img_name.append(path[38:64])
        elif len(path) == 69 :
            img_name.append(path[39:65])
        else :
            img_name.append(path[40:66])
    handle = []
    for name in img_name :
        n1 = 13
        a1 = name.replace(name[n1],':')
        n2 = 16
        a2 = a1.replace(name[n2],':')
        handle.append(a2)
    return handle 


def Extract_TimeLabel() :
    data = []
    count = 0 
    for sub_ in range(1,17 + 1 ) :
        Sub = 'Subject' + str(sub_)
        for act_ in range(1,11+1):
            Act = 'Activity'+ str(act_)
            for trial_ in range(1,3+1):
                Trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                else :
                    path = Sub + '/' + Act + '/' + Trial + '/' + Sub + Act + Trial + '.csv'

                csv = pd.read_csv(path,skiprows=1)
                count+= 1
                data.append(csv)
                if count % 100 == 0 :
                    print('path : {} . Shape : ({},{})'.format(path, csv.shape[0], csv.shape[1]))
    final = pd.concat(data)
    index = final.index
    date =[]
    for i in index :
        date.append(i[0])        
    label = final.iloc[:, -1].values
    Time_Label = pd.DataFrame(label , index = date)
    return Time_Label



def ShowImage(ImageList, nRows = 1, nCols = 2, WidthSpace = 0.00, HeightSpace = 0.00):
    from matplotlib import pyplot as plt 
    import matplotlib.gridspec as gridspec
    gs = gridspec.GridSpec(nRows, nCols)     
    gs.update(wspace=WidthSpace, hspace=HeightSpace) # set the spacing between axes.
    plt.figure(figsize=(20,20))
    for i in range(len(ImageList)):
        ax1 = plt.subplot(gs[i])
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.set_aspect('equal')

        plt.subplot(nRows, nCols,i+1)

        image = ImageList[i].copy()
        if (len(image.shape) < 3):
            plt.imshow(image, plt.cm.gray)
        else:
            plt.imshow(image)
        plt.title("Image " + str(i))
        plt.axis('off')

    plt.show()
    
    
def ResizeImage(IM, DesiredWidth, DesiredHeight):
    OrigWidth = float(IM.shape[1])
    OrigHeight = float(IM.shape[0])
    Width = DesiredWidth 
    Height = DesiredHeight

    if((Width == 0) & (Height == 0)):
        return IM
    
    if(Width == 0):
        Width = int((OrigWidth * Height)/OrigHeight)

    if(Height == 0):
        Height = int((OrigHeight * Width)/OrigWidth)

    dim = (Width, Height)
    resizedIM = cv2.resize(IM, dim, interpolation = cv2.INTER_NEAREST) 
    return resizedIM

In [6]:
Time_Label = Extract_TimeLabel()
Time_Label.head()

path : Subject4/Activity1/Trial1/Subject4Activity1Trial1.csv . Shape : (183,43)
path : Subject7/Activity1/Trial2/Subject7Activity1Trial2.csv . Shape : (191,43)
----------------------------NULL---------------------------------
----------------------------NULL---------------------------------
path : Subject10/Activity2/Trial2/Subject10Activity2Trial2.csv . Shape : (180,43)
path : Subject13/Activity2/Trial3/Subject13Activity2Trial3.csv . Shape : (185,43)
path : Subject16/Activity3/Trial1/Subject16Activity3Trial1.csv . Shape : (169,43)


,0
2018-07-04T12:04:17.738369,7
2018-07-04T12:04:17.790509,7
2018-07-04T12:04:17.836632,7
2018-07-04T12:04:17.885262,7
2018-07-04T12:04:17.945423,7


In [7]:
cwd = os.getcwd()
cwd

'/home/tf/HAR-UP/dataset'

In [8]:
nan = np.load('Background_rm/3D data/Nan_index.npy',allow_pickle = True)
set_nan = set(nan)
len(set_nan)

36565

In [9]:



def load_raw_img(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam  ):
    IMG = []
    count = 0
    name_img = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)
        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):
                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                for cam_ in range(start_cam , end_cam + 1) :
                    cam = 'Camera'+ str(cam_)

                    with ZipFile( sub + '/' + act  + '/'+ trial +  '/' + sub + act  + trial + cam + '.zip', 'r') as zipObj:
                        zipObj.extractall('CAMERA/' + sub + act  + trial  + cam)

                    for root, dirnames, filenames in os.walk('CAMERA/' + sub + act + trial  +cam ):
                        for filename in filenames:
                            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                                filepath = os.path.join(root, filename)
                                count += 1 
                                if count % 5000 == 0 :
                                    print('{} : {} ' .format(filepath,count))
                                if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png' :
                                    print('----------------------------NO SHAPE---------------------------------')
                                    continue
                                elif len(filepath) > 70 :
                                    print(' {} : Invalid image'.format(filepath))
                                    continue 
                                elif filename in set_nan :
                                    print('Remove Ind xxxxxxxxxxxxxxxxxxx ')
                                    continue
                                name_img.append(filepath)
#                                 print('read file')
#                                 img = cv2.imread(filepath, 0) 
                                img = Image.open(filepath)
                                img = np.array(img).astype('uint8')
#                                 resized = ResizeImage(img, DesiredWidth, DesiredHeight)
                                IMG.append(img)
                                
                    shutil.rmtree('CAMERA/'+ sub + act + trial + cam)
    return IMG , name_img



In [10]:

def background_Extraction(list_imgs, list_names) :
    assert len(list_imgs) == len(list_names)
    ref_size = 512
#     ref_size = 600
    # define image to tensor transform
    im_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]
    )
    # create MODNet and load the pre-trained ckpt
    modnet = MODNet(backbone_pretrained=False)
    modnet = nn.DataParallel(modnet)
    modnet.load_state_dict(torch.load('MODNet/pretrained/modnet_photographic_portrait_matting.ckpt',  map_location=torch.device('cpu')))
    modnet.eval()

    # inference images
#     im_names = list_names
#     for im_name in im_names:

    list_bg_imgs = []
    for i in range(len(list_names)):
        if i % 500 == 0 :
            print('Process image: {} {}'.format(list_names[i] , i))

        # read image
#         im = Image.open(os.path.join(args.input_path, im_name))
        # unify image channels to 3
        im = np.asarray(list_imgs[i])
        if len(im.shape) == 2:
            im = im[:, :, None]
        if im.shape[2] == 1:
            im = np.repeat(im, 3, axis=2)
        elif im.shape[2] == 4:
            im = im[:, :, 0:3]
        # convert image to PyTorch tensor
        im = Image.fromarray(im , 'RGB')
        im = im_transform(im)
        # add mini-batch dim
        im = im[None, :, :, :]
        # resize image for input
        im_b, im_c, im_h, im_w = im.shape
        if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
            if im_w >= im_h:
                im_rh = ref_size
                im_rw = int(im_w / im_h * ref_size)
            elif im_w < im_h:
                im_rw = ref_size
                im_rh = int(im_h / im_w * ref_size)
        else:
            im_rh = im_h
            im_rw = im_w
        im_rw = im_rw - im_rw % 32
        im_rh = im_rh - im_rh % 32
        im = F.interpolate(im, size=(im_rh, im_rw), mode='area')
        # inference
        _, _, matte = modnet(im.cpu(), True)
        # resize and save matte
        matte = F.interpolate(matte, size=(im_h, im_w), mode='area')
        matte = matte[0][0].data.cpu().numpy()
        matte = Image.fromarray(((matte * 255).astype('uint8')), mode='L')
        list_bg_imgs.append(matte)
#         matte_name = im_name
#         Image.fromarray(((matte * 255).astype('uint8')), mode='L').save(os.path.join(output_path, matte_name))
    return list_bg_imgs

def foreground_Extraction(image, matte):
    # calculate display resolution
#     w, h = image.shape[1], image.shape[0]
#     rw, rh = 800, int(h * 800 / (3 * w))

    # obtain predicted foreground
    image = np.asarray(image)
    if len(image.shape) == 2:
        image = image[:, :, None]
    if image.shape[2] == 1:
        image = np.repeat(image, 3, axis=2)
    elif image.shape[2] == 4:
        image = image[:, :, 0:3]
    matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
    foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
    foreground = np.uint8(foreground)

    return foreground


def backfor_Extraction(list_images, list_names):
    list_matte = background_Extraction(list_images, list_names)
    
    list_bg = []
    list_fg = []
    
    for i in range(len(list_images)):
        
        fg_img = np.array(foreground_Extraction(list_images[i], list_matte[i])).astype('uint8')
        resized_fg = ResizeImage(fg_img, DesiredWidth = 32, DesiredHeight = 32)
        list_fg.append(resized_fg)
        bg_img =  np.array(list_matte[i].convert('L')).astype('uint8')
        resized_bg = ResizeImage(bg_img, DesiredWidth = 32, DesiredHeight = 32)
        list_bg.append(resized_bg)
        
    return list_bg, list_fg
    

# Image 1

## Background & Foreground

In [12]:

list_list_bg = []
list_list_fg = []
list_list_name = []
list_list_img = []


start_sub = 1
end_sub = 3  # set this to 18 for entire dataset 

for cur_sub in range(start_sub, end_sub) :
    print('Start SUB {}'.format(cur_sub))
    start_act = 1
    end_act = 6
    start_cam = 1 
    end_cam = 1
    img_1 , first_name  = load_raw_img(cur_sub, cur_sub ,start_act  , end_act ,  start_cam ,  end_cam  )
    first_bg,first_fg = backfor_Extraction(img_1, first_name)
    del img_1
#     -----------------------------------------------------------------------
    start_act = 7
    end_act = 11
    img_1 , second_name  = load_raw_img(cur_sub, cur_sub ,start_act  , end_act ,  start_cam ,  end_cam  )
    second_bg, second_fg = backfor_Extraction(img_1, second_name)
    del img_1
#     -----------------------------------------------------------------------
    list_list_bg.append(np.concatenate((first_bg,second_bg)))
    list_list_fg.append(np.concatenate((first_fg,second_fg)))
    list_list_name.append(np.concatenate((first_name,second_name)))
    
    del first_bg, second_bg, first_fg, second_fg, first_name, second_name

Start SUB 1
CAMERA/Subject1Activity6Trial2Camera1/2018-07-04T12_12_01.821800.png : 5000 
Process image: CAMERA/Subject1Activity1Trial1Camera1/2018-07-04T12_04_23.668937.png 0
Process image: CAMERA/Subject1Activity1Trial3Camera1/2018-07-04T12_05_00.306640.png 500
Process image: CAMERA/Subject1Activity2Trial3Camera1/2018-07-04T12_06_03.442115.png 1000
Process image: CAMERA/Subject1Activity3Trial2Camera1/2018-07-04T12_06_41.607355.png 1500
Process image: CAMERA/Subject1Activity4Trial2Camera1/2018-07-04T12_07_49.290398.png 2000
Process image: CAMERA/Subject1Activity5Trial2Camera1/2018-07-04T12_08_46.569890.png 2500
Process image: CAMERA/Subject1Activity6Trial1Camera1/2018-07-04T12_09_28.517498.png 3000
Process image: CAMERA/Subject1Activity6Trial1Camera1/2018-07-04T12_10_15.737035.png 3500
Process image: CAMERA/Subject1Activity6Trial2Camera1/2018-07-04T12_11_59.117506.png 4000
Process image: CAMERA/Subject1Activity6Trial2Camera1/2018-07-04T12_12_19.429133.png 4500
Process image: CAMERA/Sub

In [16]:
all_bg = np.concatenate(list_list_bg)
all_fg = np.concatenate(list_list_fg)
all_name = np.concatenate(list_list_name)

In [178]:
path = 'Background_rm/bg_rm_data_1/data_bg_1.npy'
np.save(path, all_bg)

path = 'Background_rm/bg_rm_data_1/data_fg_1.npy'
np.save(path, all_fg)

path = 'Background_rm/bg_rm_data_1/name_bg_1.npy'
np.save(path, all_name)

## Label 

In [61]:
for i in range(len(all_name)):
    k = all_name_c[i][-30:-4]
    k = k.replace(k[13],':')
    k = k.replace(k[16],':')
    all_name[i] = k 
    
all_name


array(['2018-07-04T12:04:23.668937', '2018-07-04T12:04:23.299447',
       '2018-07-04T12:04:26.211166', ..., '2018-07-11T14:03:15.018326',
       '2018-07-11T14:03:31.557384', '2018-07-11T14:03:39.859927'],
      dtype='<U69')

In [ ]:
label_fg_1 = Time_Label.loc[all_name.tolist()].values.flatten().tolist()
path = 'Background_rm/bg_rm_data_1/label_1.npy'
np.save(path, label_fg_1)

In [ ]:
del all_bg, all_fg, all_name

# Image 2

## Background & Foreground

In [ ]:
list_list_bg = []
list_list_fg = []
list_list_name = []
list_list_img = []


start_sub = 1
end_sub = 3    # set this to 18 for entire dataset 

for cur_sub in range(start_sub, end_sub) :
    print('Start SUB {}'.format(cur_sub))
    start_act = 1
    end_act = 6
    start_cam = 2
    end_cam = 2
    img_1 , first_name  = load_raw_img(cur_sub, cur_sub ,start_act  , end_act ,  start_cam ,  end_cam  )
    first_bg,first_fg = backfor_Extraction(img_1, first_name)
    del img_1
#     -----------------------------------------------------------------------
    start_act = 7
    end_act = 11
    img_1 , second_name  = load_raw_img(cur_sub, cur_sub ,start_act  , end_act ,  start_cam ,  end_cam  )
    second_bg, second_fg = backfor_Extraction(img_1, second_name)
    del img_1
#     -----------------------------------------------------------------------
    list_list_bg.append(np.concatenate((first_bg,second_bg)))
    list_list_fg.append(np.concatenate((first_fg,second_fg)))
    list_list_name.append(np.concatenate((first_name,second_name)))
    
    del first_bg, second_bg, first_fg, second_fg, first_name, second_name

In [ ]:
all_bg = np.concatenate(list_list_bg)
all_fg = np.concatenate(list_list_fg)
all_name = np.concatenate(list_list_name)

In [ ]:
path = 'Background_rm/bg_rm_data_2/data_bg_2.npy'
np.save(path, all_bg)

path = 'Background_rm/bg_rm_data_2/data_fg_2.npy'
np.save(path, all_fg)

path = 'Background_rm/bg_rm_data_2/name_bg_2.npy'
np.save(path, all_name)

## Label 

In [ ]:
for i in range(len(all_name)):
    k = all_name_c[i][-30:-4]
    k = k.replace(k[13],':')
    k = k.replace(k[16],':')
    all_name[i] = k 
    
all_name

In [ ]:
label_fg_2 = Time_Label.loc[all_name.tolist()].values.flatten().tolist()
path = 'Background_rm/bg_rm_data_2/label_2.npy'
np.save(path, label_fg_2)

In [ ]:
del all_bg, all_fg, all_name